# Metastore and Catalog

Azure Spark comes with Hive MetaStore (HMS) pre-configured. The HMS is shared across all the pools of the same Synapse workspace.
You can leverage it to **define and manage the structure** on top of 
semi-structured data like file with the the helo of well know DBMS concepts of databases, tables, columns etc. In another words, you 
can use it define, persist metadata permanenly and manage its evolution. Persistence means the metadata survives cluster 
restarts, available across sessions and across different clusters.

```
1. SparkSession allows you to store subset of this metadata (createOrReplaceXXX() methods) but it is only available in that session scope 
and is lost after the session is closed.
````

When you want to persist the Hive catalog metadata outside of the workspace, and share catalog objects with other computational engines 
outside of the workspace, such as HDInsight and Azure Databricks, you can connect to an external Hive Metastore as explained in this [Use external Hive Metastore for Synapse Spark Pool article](https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-external-metastore). 


Metadata is (but not limited to) 

- databases
- tables
- views
- schema
    - columns names
    - data types
    - partitions
    - comments
    - Statistics (used by catalyst optimizer)

You will explore each of these in the this notebook.

## Catalog type

In [1]:
println("Catalog type: "+spark.conf.get("spark.sql.catalogImplementation"))
println("Metastore version: "+spark.conf.get("spark.sql.hive.metastore.version"))
println()
println("Default impl: "+spark.sharedState.externalCatalog)
println("Default impl delegate: "+spark.sharedState.externalCatalog.unwrapped)

## Data Warehouse location

Synapse stores the data in the `warehouse` folder of the storage account created as part of the Synapse workspace creation. 
You can get the absolute path using the `spark.sql.warehouse.dir` property.

- A database is represented by a folder with `.db` suffix at the end
- A table is represented by a folder
- Actual data is stored in files with the format (Parquet, Delta, CSV etc.) the user has chosen

In [2]:
println(spark.conf.get("spark.sql.warehouse.dir"))

## Metastore database

As you have seen above, Synapse uses Hive metastore to store metadata. The underlying RDBMS is Azure SQL. 
It can be inferred from the following properties.

In [3]:
spark.conf
    .getAll
    .filter{case (k,v)=> k.contains("spark.hadoop.javax.jdo")}
    .foreach{ case(k, v) => println(k+" = "+v)}

## What is spark.sql.catalog.spark_catalog?

This has to do with different implementations of different versions of the API and not important. Included here for completeness. 
Refer https://www.waitingforcode.com/apache-spark-sql/pluggable-catalog-api/read for more details.

In [4]:
println("Default catalog: "+spark.conf.get("spark.sql.defaultCatalog"))
println(spark.conf.get("spark.sql.catalog.spark_catalog"))

## Explore the catalog

You have 2 choices to interact with the catalog 

1. `spark.catalog` object
2. Spark SQL

### spark.catalog object

The `org.apache.spark.sql.catalog.Catalog` instance, accessible through `spark.catalog`, is used to interact with the metastore programatically.

In [5]:
val databasesDf=spark.catalog.listDatabases
display(databasesDf)

Grab the database location from above and explore the underlying folder structure.

In [6]:
%fs ls abfss://tlfs@syncdldev01.dfs.core.windows.net/synapse/workspaces/syn-tccc-cdl-use2-dev-01/warehouse/vbc.db

In [7]:
{//Introduce a code block so that the variables are not printed in the output
    import com.microsoft.spark.notebook.msutils.MSFileInfo

    val tables : Array[MSFileInfo] = mssparkutils.fs.ls("/synapse/workspaces/syn-tccc-cdl-use2-dev-01/warehouse/vbc.db")
    val tableOne = tables(0)
    println("Table name: "+tableOne.name)
    println("Is Directory: "+tableOne.isDir)
    println("Is file: "+tableOne.isFile)

    //List the files of the table
    val tableDetails : Array[MSFileInfo] = mssparkutils.fs.ls("/synapse/workspaces/syn-tccc-cdl-use2-dev-01/warehouse/vbc.db/vbc_sum_view_mth_brd_pkg_gt_wky")
    tableDetails.foreach{ f=> println("\tFile: "+f.name) }
}

In [8]:
val tablesDf = spark.catalog.listTables(dbName = "vbc")
display(tablesDf)

### Spark SQL

In [9]:
%%sql
SHOW DATABASES

In [10]:
%%sql 
SHOW TABLES FROM vbc

In [11]:
%%sql
SHOW TABLE EXTENDED FROM vbc LIKE 'vbc_sum_view_mth_brd_pkg_gtt';

## Tables

In [12]:
%%sql
--Clean up
DROP DATABASE IF EXISTS test CASCADE;

-- Recreate
CREATE DATABASE IF NOT EXISTS test;

Tables and views can be virtual if needed allowing decopuling of data & meatadata. You can 

- Insert or save data to it (managed tables) or
- Associate the path of an already existing file (External or unmanaged tables) `createExternalTable()`

In [13]:
//Create an emptry table without any data
import org.apache.spark.sql.types.{IntegerType,StringType,StructType,StructField}
val simpleSchema = StructType(Array(
    StructField("id", IntegerType, true),
    StructField("firstname",StringType,true)
  ))

val emptyTableDf = spark.catalog.createTable(tableName="test.empty_table", source="parquet", schema=simpleSchema, options=Map.empty[String, String])
emptyTableDf.show()

Synapse has [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/) package pre-installed. 
You will use [NYC Green Taxi trip records](https://azure.microsoft.com/en-us/services/open-datasets/catalog/nyc-taxi-limousine-commission-green-taxi-trip-records/) 
data in this notebook.

In [14]:
// Load nyc green taxi trip records from azure open dataset
val blob_account_name = "azureopendatastorage"

val nyc_blob_container_name = "nyctlc"
val nyc_blob_relative_path = "green"
val nyc_blob_sas_token = ""

val nyc_wasbs_path = f"wasbs://$nyc_blob_container_name@$blob_account_name.blob.core.windows.net/$nyc_blob_relative_path"
spark.conf.set(f"fs.azure.sas.$nyc_blob_container_name.$blob_account_name.blob.core.windows.net",nyc_blob_sas_token)

val nyc_tlc_df = spark.read.parquet(nyc_wasbs_path)

In [15]:
println("Num partitions: "+nyc_tlc_df.rdd.getNumPartitions)
println("Records in each partition (Partition#, Record count)")

nyc_tlc_df.rdd.mapPartitionsWithIndex{
                  // 'index' represents the Partition No, 'iterator' to iterate through all elements in the partition
                         (index, iterator) => {
                           println("Called in Partition -> " + index)
                           val numElements = iterator.length
                           // In a normal user case, we will do the
                           // the initialization(ex : initializing database)
                           // before iterating through each element
                           Array((index, numElements)).iterator
                        }
        }
        .collect
        .foreach(x => println(x))

In [16]:
nyc_tlc_df.printSchema

### Managed Tables

A managed table is a Spark SQL table for which Spark manages both the data and the metadata. A global managed table is available 
across all clusters. When you drop the table both data and metadata gets dropped.

In [17]:
//Save data to managed table
nyc_tlc_df.write.saveAsTable("test.nyc_green_taxi_trips")

//The schema from dataframe is used to create metadata
display(spark.sql("SHOW TABLE EXTENDED FROM test LIKE 'nyc_green_taxi_trips'"))

### External a.k.a UnManaged Tables

Spark manages the metadata, while you control the data location. As soon as you add `path` option in dataframe 
writer or `location` keyword to `CREATE TABLE` statement it will be treated as global external/unmanaged table. 
When you drop table only metadata gets dropped. A global unmanaged/external table is available across all clusters.

External tables are created 

1. When you use EXTERNAL keyword and specify LOCATION or 
2. LOCATION alone as part of CREATE TABLE

**Differences**

- When you drop a Managed Table, it will delete metadata from metastore as well as data. However, when you drop External Table, 
only metadata will be dropped, not the data. Typically you use External Table when same dataset is processed by multiple frameworks 
such as Synapse Serverless, Synapse SQL Pool (as external table/view), Spark, Pandas etc.
- You cannot run TRUNCATE TABLE command against External Tables.

You will use the Diabetes open data set for external table test.

```
In general CREATE TABLE is creating a “pointer”, and you need to make sure it points to something existing. 
An exception is file source such as parquet, json. If you don’t specify the LOCATION, Spark will create a default table location for you.
```



In [18]:
//Print the diabetes open dataset Blob path so that you can use `fs` magic to copy data to your synapse storage account
val diab_blob_container_name = "mlsamples"
val diab_blob_relative_path = "diabetes"
val diab_blob_sas_token = ""

val diab_wasbs_path = f"wasbs://$diab_blob_container_name@$blob_account_name.blob.core.windows.net/$diab_blob_relative_path"
println(diab_wasbs_path)

In [19]:
%%sql
-- Create table with only 3 columns even though the parquet file has 11 columns
CREATE EXTERNAL TABLE IF NOT EXISTS test.diabetes
(
    AGE	bigint COMMENT 'Age of the person',
    BMI double COMMENT 'Body mass index (weight in kg/(height in m)^2)',
    BP double COMMENT 'Diastolic blood pressure (mm Hg)'
)
USING PARQUET
LOCATION 'wasbs://mlsamples@azureopendatastorage.blob.core.windows.net/diabetes'
COMMENT 'The Diabetes dataset has 442 samples with 10 features';

-- Query metadata
DESCRIBE FORMATTED test.diabetes;

-- Query for data
SELECT COUNT(*) FROM test.diabetes;

#### Create external table from a dataframe.

```
dataframe.write.option('path', "<your-storage-path>").saveAsTable("my_table")
```

## Views

### Permanent Views

- The view definition is recorded in the underlying metastore. 
- You can only create permanent view on **global managed table or global unmanaged table**. 
- Not allowed to create a permanent view on top of any temporary views or dataframe. 

```
Note: Permanent views are only available in SQL API — not available in dataframe API
```

Persist a dataframe as permanent view. 

In [20]:
%%sql
-- Create a view on top of managed table
CREATE VIEW IF NOT EXISTS
    test.view_nyc_green_taxi_trips 
AS 
    SELECT * FROM test.nyc_green_taxi_trips;

-- Query total row count from View
SELECT COUNT(1) FROM test.view_nyc_green_taxi_trips;

-- Query total row count from Table
SELECT COUNT(1) FROM test.nyc_green_taxi_trips;

-- Show the views
SHOW VIEWS FROM test;

### Temporary Views

These are single or multiple session scoped. The meatadata will be lost as soon as the underlying sessions are closed.

#### Local Temp Views

Temportary views are used generally 

- to share data between differnt languages in a notebook
- to support SQL on DataFrames. Using Spark SQL requires metadata (schema, partitions) to be available. We can use the approach
mentioned under 'Tables` section above to create this metadata but it is unnecessary if the scope of usage is local to current
processing. In this scenarion a temporary view is useful.

You can use `df.createOrReplaceTempView` to create the temp view. A temporary view is 

- Spark session scoped. 
- A local view is not accessible from other notebooks in Synapse
- Not registered in the metastore.

##### Create

Spark SQL syntax that can replace the following 2 Scala statements: 

`CREATE TEMP VIEW temp_view_green_taxi_trips AS SELECT * FROM test.nyc_green_taxi_trips;`

In [21]:
val nycGreenTaxiDf = spark.sql("SELECT * FROM test.nyc_green_taxi_trips")

//Create the view in "default" database. You can create temp views in any database but it will not be permanent
nycGreenTaxiDf.createOrReplaceTempView("temp_view_green_taxi_trips")

##### Query

In [22]:
display(spark.table("temp_view_green_taxi_trips").limit(1))

##### List

In [23]:
%%sql

SHOW VIEWS;

#### Global Temp Views

- Spark `application` scoped
- Are tied to a system preserved temporary database `global_temp`. 
- Can be shared across different spark sessions (or if using databricks notebooks, then shared across notebooks).
- Not supported in Synapse Spark.

You can use `df.createOrReplaceGlobalTempView("my_global_view")` and can be accessed as `spark.read.table("global_temp.my_global_view")`

##### Create

In [24]:
nycGreenTaxiDf.createOrReplaceGlobalTempView("global_temp_view_green_taxi_trips")

##### Query

In [25]:
val globalTempViewTaxiDf = spark.read.table("global_temp.global_temp_view_green_taxi_trips")
display(globalTempViewTaxiDf.limit(1))

##### List

In [26]:
%%sql
-- List all views in global temp view database. 
-- Note that the command also lists local temporary views regardless of a given database.
SHOW VIEWS FROM global_temp;

## Partitions

You will use NYC Yello Taxi (1.5B rows, 50 GB) to demonstrate the Partitions.

### Managed Table

#### Create a paritioned "Managed" table

In [27]:
// Azure storage access info
val blob_account_name = "azureopendatastorage"
val blob_container_name = "nyctlc"
val blob_relative_path = "yellow"
val blob_sas_token = "r"

// Allow SPARK to read from Blob remotely
val wasbs_path = f"wasbs://$blob_container_name@$blob_account_name.blob.core.windows.net/$blob_relative_path"
spark.conf.set(f"fs.azure.sas.$blob_container_name.$blob_account_name.blob.core.windows.net",blob_sas_token)
println("Remote blob path: " + wasbs_path)

// SPARK read parquet, note that it won't load any data yet by now
val nycYellowTaxiDf = spark.read.parquet(wasbs_path)

println("Num partitions: "+nycYellowTaxiDf.rdd.getNumPartitions)

In [28]:
// Save as managed table
nycYellowTaxiDf.write.partitionBy("puYear", "puMonth").saveAsTable("test.nyc_yellow_taxi_trips_partitioned")

//The schema from dataframe is used to create metadata
display(spark.sql("SHOW TABLE EXTENDED FROM test LIKE 'test.nyc_yellow_taxi_trips_partitioned'"))

#### Check meatadata for partition info

In [29]:
%%sql
DESCRIBE TABLE EXTENDED test.nyc_yellow_taxi_trips_partitioned;

-- Show all paritions
SHOW PARTITIONS test.nyc_yellow_taxi_trips_partitioned;

--Show details on specific partition
SHOW PARTITIONS test.nyc_yellow_taxi_trips_partitioned PARTITION (puYear="2012");

### External a.k.a UnManaged Table

#### Create partitioned dataset on storage account

In [30]:
// Repartition and write the partitions to ADLS under specified folder
var pathExists = true;

try{
    mssparkutils.fs.ls("/poc/nyc_yellow_taxi_trips_partitioned")
} catch {
    case ex: Exception => {pathExists = false}
}

if(!pathExists) {
    nycYellowTaxiDf.write.partitionBy("puYear", "puMonth").parquet("/poc/nyc_yellow_taxi_trips_partitioned")
}

Verify that the data is written to partitions and that we can read it.

In [31]:
spark.read
    .parquet("/poc/nyc_yellow_taxi_trips_partitioned").limit(5)
    .show()

#### Create External Table
Add meta information to catalog by creating a table with "path" i.e. external or unmanaged table

In [32]:
//Create external table
spark.catalog.createTable("test.external_nyc_yellow_taxi_trips_partitioned",
                path="/poc/nyc_yellow_taxi_trips_partitioned",
                source="parquet")

#### Check metadata for partition info
The `DESCRIBE` commands output shows us the Partiotion columns have been identified by and persisted in the catalog but the `SELECT` 
queries still doesn't work.

In [33]:
%%sql
DESCRIBE TABLE EXTENDED test.external_nyc_yellow_taxi_trips_partitioned;


In [34]:
%%sql

SELECT * FROM test.external_nyc_yellow_taxi_trips_partitioned;
SHOW PARTITIONS test.external_nyc_yellow_taxi_trips_partitioned;

#### Recover (Fix) partition details into catalog
As you can see from the above output `No data available`, the data in the path `/poc/nyc_yellow_taxi_trips_partitioned` is visible to 
file based readers but invisible while reading through the catalog using **Spark SQL**. If the external table is not partitioned, this is 
not an issue. but for partitioned datasets, this doesn't work unless we do a `recover` operation. 

```
We can recover partitions by running MSCK REPAIR TABLE using spark.sql or by invoking spark.catalog.recoverPartitions.
```

In [35]:
// When we use createTable to create partitioned table, we have to recover partitions so that partitions are visible.
spark.catalog.recoverPartitions("test.external_nyc_yellow_taxi_trips_partitioned")

Spark SQL Command can be used for recovering partitions as shown below.

In [36]:
%%sql

MSCK REPAIR TABLE test.external_nyc_yellow_taxi_trips_partitioned;

In [37]:
%%sql

SELECT * FROM test.external_nyc_yellow_taxi_trips_partitioned LIMIT 2;
show PARTITIONS test.external_nyc_yellow_taxi_trips_partitioned;

In [38]:
%%sql

ANALYZE TABLE test.external_nyc_yellow_taxi_trips_partitioned COMPUTE STATISTICS;
DESCRIBE EXTENDED test.external_nyc_yellow_taxi_trips_partitioned;

## Statistics

Statistics are supported for the following only:

Hive Metastore tables for which ANALYZE TABLE <tableName> COMPUTE STATISTICS noscan has been executed
File-based data source tables for which the statistics are computed directly on the files of data

### Check stats using `explain(...)`

Since Spark 3.0 you can use the `explain()` method to display the stats and see it not only for the table but for the actual query 
that we want to run. This can be done by using the new `mode` argument of the explain function. This is going to show us two query plans, 
namely the physical plan and also the optimized logical plan. The logical plan now contains the information about the statistics as 
you can see in the output of following command (Look for `Statistics` attribute under `Optimized Logical Plan`).

In [39]:
spark.table("vbc.vbc_sum_view_mth_brd_pkg_gtt").explain(mode="cost")

### Compute and check stats using Spark SQL

In [40]:
%%sql
ANALYZE TABLE test.nyc_green_taxi_trips COMPUTE STATISTICS;
DESCRIBE EXTENDED test.nyc_green_taxi_trips;

## Additional Reading

- [Types of Apache Spark tables and views](https://medium.com/@subashsivaji/types-of-apache-spark-tables-and-views-f468e2e53af2)
- [Spark Statistics Explained](https://towardsdatascience.com/statistics-in-spark-sql-explained-22ec389bf71b)
- [Spark DDL SQL Reference](https://spark.apache.org/docs/latest/sql-ref-syntax.html)
- [CREATE TABLE syntax](https://spark.apache.org/docs/latest/sql-ref-syntax-ddl-create-table.html)
- [SHOW VIEWS syntax](https://spark.apache.org/docs/latest/sql-ref-syntax-aux-show-views.html)
- [Azure Open Datasets](https://docs.microsoft.com/en-us/azure/open-datasets/)
- [Microsoft Research Open Data](https://msropendata.com/)